In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()




In [6]:
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')

In [8]:
# Create options for the dropdown list
site_options = [{'label': 'All Sites', 'value': 'ALL'}]
site_options.extend([{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()])

# Define function to generate scatter chart
def generate_scatter_chart(df):
    fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig

In [10]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                             options=site_options,
                                             value='ALL',
                                             placeholder='Select a Launch Site',
                                             searchable=True),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),

                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                 min=0,
                                                 max=10000,
                                                 step=1000,
                                                 value=[min(spacex_df['Payload Mass (kg)']),
                                                        max(spacex_df['Payload Mass (kg)'])]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def update_success_pie_chart(site):
    if site == 'ALL':
        values = spacex_df['class'].value_counts()
        title = 'Total Success Launches By Site'
    else:
        df_filtered = spacex_df[spacex_df['Launch Site'] == site]
        values = df_filtered['class'].value_counts()
        title = f"Total Success Launches for site {site}"
    fig = {
        'data': [{'values': values,
                  'labels': ['Failure', 'Success'],
                  'type': 'pie'}],
        'layout': {'title': title,
                   'legend': {'orientation': 'h', 'xanchor': 'center', 'x': 0.5},
                   'plot_bgcolor': '#F2DFCE',
                   'paper_bgcolor': '#F2DFCE'}
    }
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs,
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def update_success_payload_scatter_chart(site, payload):
    if site == 'ALL':
        df_filtered = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload[1])]
        fig = generate_scatter_chart(df_filtered)
    else:
        df_filtered = spacex_df[(spacex_df['Launch Site'] == site) &
                                (spacex_df['Payload Mass (kg)'] >= payload[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload[1])]
        fig = generate_scatter_chart(df_filtered)
        
    return fig

# Run the app
if __name__ == '__main__':
    #app.run_server()
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
